<a href="https://colab.research.google.com/github/nbrunella19/CIINTI/blob/main/CIINTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerias

In [1]:
import io
import glob
import os
import sys                                                                # to indicate errors and exceptions
import re
import traceback
import cmath  # Use cmath for complex roots
import numpy as np
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
from google.colab import drive
from io import StringIO
from datetime import datetime
from google.colab import files
from IPython.display import FileLink
from IPython.display import display
from IPython.display import clear_output


In [2]:
file_path ='/content/drive/MyDrive/Colab Notebooks/Mediciones/MEDICIONES_DE_R.mea'
file_path_t= '/content/drive/MyDrive/Colab Notebooks/Mediciones/MEDICIONES_DE_T.dat'
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Funciones


In [3]:
# Función para seleccionar un archivo
def seleccionar_archivo():
    # Muestra un cuadro de diálogo para cargar un archivo
    uploaded = files.upload()

    # Obtiene la ruta del archivo cargado
    ruta_al_archivo = list(uploaded.keys())[0]

    # Crea un enlace para abrir el archivo
    file_path_i = FileLink(ruta_al_archivo)
    # Muestra el enlace
    return (file_path_i)

In [4]:
def openFile(path2file):
    '''
    openFile
    ___________________
    open the file, returns a list of the lines of path2file.
    It is assumed that path2file is a .mea file (text based)
    out = lines of file (strings)
    input= path
    '''
    try:
        with open(path2file, mode='r') as f:  # to ensure it closes correclty
            lines = f.readlines()
            return lines
    except Exception as exception:
        print("There was a problem opening {}".format(path2file))
        print(exception)
        raise

def read_mea_file(lines_of_mea_file, ln_preface = 81):
    '''
    The function reads and arrange the data list of the file called by openFile.
    Inputs:
        lines_of_mea_file: list of str lines read by openFile()
        ln_preface(optional, default=  81): number of lines in the preface of the measurments file. They appear in all sets of measurements.
    Outputs:
        qlistInit: list containing the set measurement prefaces, eg. qlistInit[2]= preface of measurement 3
        qlist: list containing the set measurements, eg. qlist[2]= measurement set number 3
        qlistFinal: list containing the result resume of the measurement set, eg. qlistFinal[2]= resume of measurement set 3
    To process it is preferable to have all measurements together in the same file.
    Note that it doesn´t matter if the measurements have differet number of points.
    '''
    chunks = []
    qlist = []
    chunksInit =[]
    qlistInit = []
    chunksFinal = []
    qlistFinal = []
    nn = 0
    counter = 0
    ii = 0
    for line in lines_of_mea_file:
        if (nn>ln_preface and line != '\n' and ii == 0):
            chunks.append(line)
            nn += 1
        elif (nn<ln_preface):
            chunksInit.append(line)
            nn +=1
        elif (nn>ln_preface and line == '\n'):
            ii = 1
            nn += 1
        elif (nn>ln_preface and ii == 1 and line != '***\n'):
            chunksFinal.append(line)
            nn += 1
        elif (line =='***\n'):
            ii = 0
            counter += 1
            nn = 0
            qlist.append(chunks)
            qlistInit.append(chunksInit)
            qlistFinal.append(chunksFinal)
            chunks = []
            chunksInit = []
            chunksFinal = []
        else:
            nn += 1
    return qlistInit, qlist, qlistFinal,counter

In [5]:
def openFile_T(path2file):
    '''
    openFile
    ___________________
    open the file, returns a list of the lines of path2file.
    It is assumed that path2file is a .dat file (text based)
    out = lines of file (strings)
    input= path
    '''
    try:
        with open(path2file, mode='r', encoding='utf-8', errors='ignore') as g:  # to ensure it closes correclty
            lines = g.readlines()
            return lines
    except Exception as exception:
        print("There was a problem opening {}".format(path2file))
        print(exception)
        raise

In [6]:
def counter_different_values(lista):
    '''
    Accounts for different types of measurement current
    '''
    different_values = set(lista)
    different_quantity = len(different_values)
    return different_quantity

#resume = counter_different_values(iList)

In [7]:
e = 20
def create_df_from_list(qlist, n):
    '''
    Create a DataFrame from a list that coming from the .txt
    '''
    df_aux = pd.DataFrame([sub.split(';') for sub in qlist[n]])
    df = df_aux.tail(e)                                               # Recorta la lista a un N° "e" (proximamente programable por el usuario)
    df.columns = ['Ratio', 'Measure', 'Time']
    return df

In [8]:
def convertir_formato(tiempo_string):
    try:
      tiempo_objeto = datetime.strptime(tiempo_string, '%H:%M:%S.%f')
      nuevo_formato = tiempo_objeto.strftime('%H:%M:%S')
    except ValueError:
      nuevo_formato=tiempo_string
    return nuevo_formato

In [9]:
#file_path= str(seleccionar_archivo())
#file_path ='/content/MEDICIONES_DE_R_CON_T (3).mea'


In [10]:
#file_path_t= str(seleccionar_archivo())
#file_path_t= '/content/MEDICIONES_DE_T. (2).dat'

# Análisis de archivo de medición de resistencia

In [11]:
lineas = openFile(file_path)
qlistInit, qlist, qlistFinal,counter= read_mea_file(lineas, ln_preface = 81)


In [12]:
df_m = [create_df_from_list(qlist, i) for i in range(counter)]         # I Create a list of DataFrame

for i in range(counter):
    df_m[i]['Time'] = df_m[i]['Time'].str.strip()                      # Le saca el a la columan Time'\n'

#df_m

In [13]:
measure_f=[]  #Lista de mediciones en formato float
time_h=[]     #Lista de hora en formato "H":"M":"S"
for i in range(counter):
      # Genero un listado de listas del tipo float
      measure_f.append([float(numero) for numero in df_m[i]["Measure"]])
      time_h.append(df_m[i]["Time"])


In [14]:
#Acá obtengo una lista de los parámetros de las mediciones

iList=[0]*counter               #Listado de corrientes
RsList=[0]*counter              #Listado de patrones
RxList=[0]*counter              #Listado de Mesurandos
MvList=[0]*counter              #Listado de Mediciones medias
StList=[0]*counter              #Listado de desvios
StartDateList=[0]*counter       #Listado de fecha de inicio
StartHoursList=[0]*counter      #Listado de horario de inicio
StopDateList=[0]*counter        #Listado de fecha de finalización
StopHoursList=[0]*counter       #Listado de horario de finalización

for i in range(counter):
  StartDate_aux= qlistInit[i][13].strip()
  StartDateList[i]= StartDate_aux[:9]
  StartHoursList[i]= StartDate_aux[-8:]

  StopDate_aux= qlistFinal[i][11].strip()
  StopDateList[i]= StopDate_aux[:9]
  StopHoursList[i]= StopDate_aux[-8:]

  iList[i]=  qlistInit[i][22].strip()
  RsList[i]= qlistInit[i][33].strip()
  RxList[i]= qlistInit[i][57].strip()
  MvList[i]= np.mean(measure_f[i])
  StList[i]= np.std(measure_f[i])

#StopDateList
#StartHoursList
#DateList
#iList,RsList,RxList
#MvList,StList

# Análisis de archivo de medición de temperatura

In [15]:
lineas_t = openFile_T(file_path_t)
#lineas_t

In [16]:
lineas_t_str = '\n'.join(lineas_t)
df_t = pd.read_csv(StringIO(lineas_t_str), delimiter=';', skiprows=1, header=None, names=['Time T', 'Mean T', 'NaN'], parse_dates=True)
df_t = df_t.drop(0, axis=0)         #Borra el primer valor
df_t = df_t.drop(columns=['NaN'])   #Borro la columna NaN que no sirve
df_t = df_t[['Mean T', 'Time T']]   #Invierto las columnas

size_df_t=df_t.shape[0]
#size_df_t

In [17]:
timetempList=[]

timetempList.append(df_t['Time T'])
timetempList = [convertir_formato(tiempo) for tiempo in timetempList[0]]

df_t['Time T']= timetempList       #Listado de tiempos detemperatura en formato H:M:S

In [18]:
def get_time_measure(df_m,index):
  row_start=0
  row_end=19
  timestart= df_m[index].iloc[row_start, df_m[index].columns.get_loc('Time')]
  timestop = df_m[index].iloc[row_end, df_m[index].columns.get_loc('Time')]
  return timestart,timestop


In [19]:
def solve_quadratic(a, b, c):
    '''
    Esta ecuación resuelve la cuadrática asumiendo que
    ax**2+bx+c=0
    output = root1, root2
    '''
    # Calculate the discriminant
    discriminant = cmath.sqrt(b**2 - 4*a*c)

    # Calculate the solutions
    root1 = (-b + discriminant) / (2*a)
    root2 = (-b - discriminant) / (2*a)

    root1 = root1.real if abs(root1.imag) < 1e-10 else float('nan')
    root2 = root2.real if abs(root2.imag) < 1e-10 else float('nan')

    return float(root1), float(root2)




In [20]:

def calculator_temp_average(timestart,timestop):
  templist=[]

  R_0 = 99.942247
  alpha=0.0039253221
  beta=-7.5009932*10**(-7)

  for i in range(size_df_t):
    time_aux = df_t.iloc[i, df_t.columns.get_loc('Time T')]
    if time_aux>=timestart and time_aux<=timestop:
      templist.append(df_t.iloc[i, df_t.columns.get_loc('Mean T')])

  R_T=np.mean(templist)

  c1 = R_0- R_T
  b1 = R_0*alpha
  a1 = R_0*beta
  T1,T2 = solve_quadratic(a1, b1, c1)
  if T2>0 and T2<35:
    Tfinal= T2
  else:
     Tfinal= T1
  return Tfinal


# Entorno gráfico

In [36]:
############################################ Opciones para el menú desplegable ############################################################
flag_lmea=0
flag_ldat=0

flag_En_Calc=0
flag_En_Graf=0

flag_Ch_Drop=0

index=0
counter_m = []

lineas = openFile(file_path)
qlistInit, qlist, qlistFinal,counter= read_mea_file(lineas, ln_preface = 81)


for i in range(counter+1):
  if(i==0):
    counter_m.append("---")
  else:
    counter_m.append(str(i))
  counter_mv = {opcion: i  for i, opcion in enumerate(counter_m)}

def handle_dropdown_change(change):
    global index,flag_Ch_Drop
    if flag_Ch_Drop==0:
      opcion_seleccionada = change.new
      index = counter_mv[opcion_seleccionada]-1
      print(f"Opción seleccionada: {opcion_seleccionada}")
      display(df_m[index])
      flag_Ch_Drop=1
                                           # Crear el menú desplegable
menu_desplegable = widgets.Dropdown(
    options=counter_m,
    value=counter_m[0],  # Valor predeterminado seleccionado
    description='Medición:')

############################################################################################################################################
################################################# Funciones para Botones  ##################################################################

def on_button1_click(b):
  global flag_En_Calc
  if flag_En_Calc==0:
    sd=np.std(measure_f[index])    # sd es el desvío estándar (standard deviation )
    am=np.mean(measure_f[index])   # am es el valor promedio medido (average measure)
    output_text1.value = "El valor medio de la medición "+str(index+1)+" es: "+str(am)
    output_text2.value = "Incertidumbre tipo a (Ua) es: "+str(sd)
    output_text3.value = "La Temperatura media (T): " + str(calculator_temp_average(*get_time_measure(df_m, index)))
    flag_En_Calc=1

def on_button2_click(b):
    global flag_En_Calc, flag_En_Graf,flag_Ch_Drop
    output_text1.value = ''
    output_text2.value = ''
    output_text3.value = ''
    flag_En_Calc=0
    flag_En_Graf=0
    flag_Ch_Drop=0
    clear_output()
    show_menu()


def on_button3_click(b):
    global index, flag_En_Graf
    if flag_En_Graf==0:
      t_x = time_h[index]
      r_y = measure_f[index]
      r_y_array = np.array(r_y, dtype=float)
      r_y_array = r_y_array - r_y_array.mean()
      plt.figure(figsize=(15,6))
      plt.xticks(rotation=45)
      display(plt.plot(t_x, r_y_array))
      flag_En_Graf=1

def on_button4_click(b):                      #Carga de .mea
    global flag_lmea
    file_path= str(seleccionar_archivo())
    flag_lmea=1                               #Flag que me indica que toque el botón


def on_button5_click(b):                      #Carga de .dat
    global flag_ldat
    file_path_t= str(seleccionar_archivo())
    flag_ldat=1

def on_button6_click(b):                      #Procesar
    global flag_lmea,flag_ldat
    if(flag_lmea==1 and flag_ldat==1):
      clear_output()
      show_menu()

def show_menu():
    display(menu_desplegable)
    display(button1)
    display(button3)
    display(button2)
    display(output_text1)
    display(output_text2)
    display(output_text3)

def show_window_1():
    display(button4)
    display(button5)
    display(button6)




############################################################################################################################################

# Crear widgets
output_text1 = widgets.Label(value='')
output_text2 = widgets.Label(value='')
output_text3 = widgets.Label(value='')


button1 = widgets.Button(description="Calcular")
button2 = widgets.Button(description="Limpiar")
button3 = widgets.Button(description="Graficar")
button4 = widgets.Button(description="Cargar *.mea")
button5 = widgets.Button(description="Cargar *.dat")
button6 = widgets.Button(description="Siguiente")


# Configurar la función a llamar cuando se hace clic en el botón
button1.on_click(on_button1_click)
button2.on_click(on_button2_click)
button3.on_click(on_button3_click)
button4.on_click(on_button4_click)
button5.on_click(on_button5_click)
button6.on_click(on_button6_click)


# Conectar la función al evento de cambio en el menú desplegable
menu_desplegable.observe(handle_dropdown_change, names='value')

# Mostrar los widgets
show_window_1()





Dropdown(description='Medición:', index=3, options=('---', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', …

Button(description='Calcular', style=ButtonStyle())

Button(description='Graficar', style=ButtonStyle())

Button(description='Limpiar', style=ButtonStyle())

Label(value='')

Label(value='')

Label(value='')

Opción seleccionada: 2


,Ratio,Measure,Time
15,0.08007427705894,8.007427705894,14:25:27
16,0.08007428192199,8.007428192199,14:25:36
17,0.08007428631657,8.007428631657,14:25:44
18,0.08007429030157,8.007429030157,14:25:53
19,0.08007429400251,8.007429400251,14:26:01
20,0.08007429736434,8.007429736434,14:26:09
21,0.08007430037401,8.007430037401,14:26:18
22,0.08007430311264,8.007430311264,14:26:26
23,0.08007430566391,8.007430566391,14:26:35
24,0.08007430802524,8.007430802524,14:26:43
